# <font color=green>6. WEB SCRAPING DO SITE ALURA MOTORS - OBTENDO OS DADOS DE TODOS OS ANÚNCIOS DE UMA PÁGINA

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
response = urlopen('https://alura-site-scraping.herokuapp.com/index.php')
html = response.read().decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')

# 6.1. Identificando as informações no HTML

In [3]:
soup.find('div', {"id": "container-cards"})

<div id="container-cards" style="height: 100%">
<div class="well card">
<div class="col-md-3 image-card">
<img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-aventador/lamborghini-aventador-2932196__340.jpg" width="220"/>
</div>
<div class="col-md-6 body-card">
<p class="txt-name inline">LAMBORGHINI AVENTADOR</p>
<p class="txt-category badge badge-secondary inline">USADO</p>
<p class="txt-motor">Motor 1.8 16v</p>
<p class="txt-description">Ano 1993 - 55.286 km</p>
<ul class="lst-items">
<li class="txt-items">► 4 X 4</li>
<li class="txt-items">► Câmera de estacionamento</li>
<li class="txt-items">► Controle de tração</li>
<li class="txt-items">► Sensor de estacionamento</li>
<li class="txt-items">...</li>
</ul>
<p class="txt-location">Belo Horizonte - MG</p>
</div>
<div class="col-md-3 value-card">
<div class="value">
<p class="txt-value">R$ 338.000</p>
</div>
</div>
</div>
<div class="well card">
<div class="col-md-3 ima

In [4]:
soup.find('div', {"id": "container-cards"}).findAll('div', class_="card")

[<div class="well card">
 <div class="col-md-3 image-card">
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-aventador/lamborghini-aventador-2932196__340.jpg" width="220"/>
 </div>
 <div class="col-md-6 body-card">
 <p class="txt-name inline">LAMBORGHINI AVENTADOR</p>
 <p class="txt-category badge badge-secondary inline">USADO</p>
 <p class="txt-motor">Motor 1.8 16v</p>
 <p class="txt-description">Ano 1993 - 55.286 km</p>
 <ul class="lst-items">
 <li class="txt-items">► 4 X 4</li>
 <li class="txt-items">► Câmera de estacionamento</li>
 <li class="txt-items">► Controle de tração</li>
 <li class="txt-items">► Sensor de estacionamento</li>
 <li class="txt-items">...</li>
 </ul>
 <p class="txt-location">Belo Horizonte - MG</p>
 </div>
 <div class="col-md-3 value-card">
 <div class="value">
 <p class="txt-value">R$ 338.000</p>
 </div>
 </div>
 </div>,
 <div class="well card">
 <div class="col-md-3 image-card">
 <img alt="

In [5]:
len(soup.find('div', {"id": "container-cards"}).findAll('div', class_="card"))

10

In [6]:
anuncios = soup.find('div', {"id": "container-cards"}).findAll('div', class_="card")

In [7]:
for anuncio in anuncios:
    print(str(anuncio) + "\n\n")

<div class="well card">
<div class="col-md-3 image-card">
<img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-aventador/lamborghini-aventador-2932196__340.jpg" width="220"/>
</div>
<div class="col-md-6 body-card">
<p class="txt-name inline">LAMBORGHINI AVENTADOR</p>
<p class="txt-category badge badge-secondary inline">USADO</p>
<p class="txt-motor">Motor 1.8 16v</p>
<p class="txt-description">Ano 1993 - 55.286 km</p>
<ul class="lst-items">
<li class="txt-items">► 4 X 4</li>
<li class="txt-items">► Câmera de estacionamento</li>
<li class="txt-items">► Controle de tração</li>
<li class="txt-items">► Sensor de estacionamento</li>
<li class="txt-items">...</li>
</ul>
<p class="txt-location">Belo Horizonte - MG</p>
</div>
<div class="col-md-3 value-card">
<div class="value">
<p class="txt-value">R$ 338.000</p>
</div>
</div>
</div>


<div class="well card">
<div class="col-md-3 image-card">
<img alt="Foto" height="155" src="ht

# 6.2. Criando uma rotina de scraping

In [8]:
# Importando bibliotecas
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

# Declarando variável cards
cards = []

# Obtendo o HTML
response = urlopen('https://alura-site-scraping.herokuapp.com/index.php')
html = response.read().decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')

# Obtendo as TAGs de interesse
anuncios = soup.find('div', {"id": "container-cards"}).findAll('div', class_="card")

# Coletando as informações dos CARDS
for anuncio in anuncios:
    card = {}
    
    # Valor
    card['value'] = anuncio.find('p', {'class': 'txt-value'}).getText()

    # Informações
    infos = anuncio.find('div', {'class': 'body-card'}).findAll('p')
    for info in infos:
        card[info.get('class')[0].split('-')[-1]] = info.get_text()

    # Acessórios
    items = anuncio.find('div', {'class': 'body-card'}).ul.findAll('li')
    items.pop()
    acessorios = []
    for item in items:
        acessorios.append(item.get_text().replace('► ', ''))
    card['items'] = acessorios
    
    # Adicionando resultado a lista cards
    cards.append(card)

    # Imagens
    image = anuncio.find('div', {'class': 'image-card'}).img
    urlretrieve(image.get('src'), './output/img/' + image.get('src').split('/')[-1])     

# Criando um DataFrame com os resultados
dataset = pd.DataFrame(cards)
dataset.to_csv('./output/data/dataset.csv', sep=';', index = False, encoding = 'utf-8-sig')
dataset

,value,name,category,motor,description,location,items,opportunity
0,R$ 338.000,LAMBORGHINI AVENTADOR,USADO,Motor 1.8 16v,Ano 1993 - 55.286 km,Belo Horizonte - MG,"[4 X 4, Câmera de estacionamento, Controle de ...",NaN
1,R$ 346.000,BMW M2,USADO,Motor 3.0 32v,Ano 2018 - 83.447 km,Belo Horizonte - MG,"[Câmera de estacionamento, Controle de estabil...",NaN
2,R$ 480.000,ALFA,USADO,Motor 1.8 16v,Ano 2004 - 19.722 km,Rio de Janeiro - RJ,"[Central multimídia, Bancos de couro, Rodas de...",NaN
3,R$ 133.000,PUECH,USADO,Motor Diesel V8,Ano 1992 - 34.335 km,São Paulo - SP,"[Bancos de couro, Freios ABS, Rodas de liga, C...",NaN
4,R$ 175.000,LAMBORGHINI MURCIELAGO,USADO,Motor 1.0 8v,Ano 1991 - 464 km,Belo Horizonte - MG,"[Central multimídia, Teto panorâmico, Sensor c...",NaN
5,R$ 239.000,ASTON MARTIN,USADO,Motor Diesel V6,Ano 2004 - 50.189 km,Belo Horizonte - MG,"[Painel digital, Controle de tração, Teto pano...",OPORTUNIDADE
6,R$ 115.000,TVR,USADO,Motor 4.0 Turbo,Ano 2014 - 17.778 km,Belo Horizonte - MG,"[4 X 4, Teto panorâmico, Central multimídia, C...",NaN
7,R$ 114.000,EXCALIBUR,USADO,Motor 3.0 32v,Ano 2009 - 81.251 km,Rio de Janeiro - RJ,"[Painel digital, Câmbio automático, Sensor de ...",NaN
8,R$ 75.000,MCLAREN,NOVO,Motor Diesel,Ano 2019 - 0 km,São Paulo - SP,"[Central multimídia, Câmera de estacionamento,...",NaN
9,R$ 117.000,TOYOTA,USADO,Motor 4.0 Turbo,Ano 1999 - 12.536 km,São Paulo - SP,"[Bancos de couro, Freios ABS, Piloto automátic...",OPORTUNIDADE


In [9]:
cards

[{'value': 'R$ 338.000',
  'name': 'LAMBORGHINI AVENTADOR',
  'category': 'USADO',
  'motor': 'Motor 1.8 16v',
  'description': 'Ano 1993 - 55.286 km',
  'location': 'Belo Horizonte - MG',
  'items': ['4 X 4',
   'Câmera de estacionamento',
   'Controle de tração',
   'Sensor de estacionamento']},
 {'value': 'R$ 346.000',
  'name': 'BMW M2',
  'category': 'USADO',
  'motor': 'Motor 3.0 32v',
  'description': 'Ano 2018 - 83.447 km',
  'location': 'Belo Horizonte - MG',
  'items': ['Câmera de estacionamento',
   'Controle de estabilidade',
   'Travas elétricas',
   'Freios ABS']},
 {'value': 'R$ 480.000',
  'name': 'ALFA',
  'category': 'USADO',
  'motor': 'Motor 1.8 16v',
  'description': 'Ano 2004 - 19.722 km',
  'location': 'Rio de Janeiro - RJ',
  'items': ['Central multimídia',
   'Bancos de couro',
   'Rodas de liga',
   'Câmera de estacionamento']},
 {'value': 'R$ 133.000',
  'name': 'PUECH',
  'category': 'USADO',
  'motor': 'Motor Diesel V8',
  'description': 'Ano 1992 - 34.335 

### Exercicio - Interando Tags e acessando Atributos

In [1]:
from bs4 import BeautifulSoup

html4 = """
    <html>
        <body>
            <h1>Fotos</h1>
            <div>
                <img alt="Foto" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/mercedes-benz-slr/mercedes-benz-slr-2925433__340.jpg">
                <img alt="Foto" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/vwbeetle/vwbeetle-836065__340.jpg">
                <img alt="Foto" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/porsche-911-gt2rs/porsche-911-gt2rs-3076518__340.jpg">
            </div>
        </body>
    </html>
"""
soup4 = BeautifulSoup(html4, 'html.parser')

In [2]:
for image in soup4('img'):
    print(image.get('src'))

https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/mercedes-benz-slr/mercedes-benz-slr-2925433__340.jpg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/vwbeetle/vwbeetle-836065__340.jpg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/porsche-911-gt2rs/porsche-911-gt2rs-3076518__340.jpg
